# Fortgeschrittene Konzepte für RNNs
## GRU-Schichten
GRU (Gated Recurrent Unit)-Schichten erfüllen diesselbe Funktion wie eine LSTM-Schicht: sie lassen den Fluss von Informationen aus weit vorangegangen Perioden gewichtet zu.

Der Unterschied ist, dass sie weniger Rechenleistung benötigen, was mit einer kleineren Repäsentationskraft bezahlt wird
 
## Recurrent Dropout
In der Forschung schien sich zunächst abzubilden, dass Dropout in RNNs eher hinderlich als hilfreich ist. Wird allerdings das gleiche Muster an Dropout auf jede Schicht angewandt, so ändert sich das. So sollte auch das gleiche Muster in allen Matrizen innerhalb einer RNN-Schicht verwendet werden. Dies lässt sich so codieren:

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
model = Sequential()
# ...
model.add(GRU(
    32, 
    dropout = 0.2,
    recurrent_dropout = 0.2,
    input_shape = (None, data.shape[-1])
))
print(model.summary())
# ...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 32)                3456      
Total params: 3,456
Trainable params: 3,456
Non-trainable params: 0
_________________________________________________________________
None


## Mehrere RNN-Schichten hintereinander
Um das zu tun, müssen alle Schichten, die nicht ganz oben liegen `return_sequences = True` aktiviert haben. Es ist ein guter Weg, um ein Netz zu erweitern, um es an die Grenzen des Overfitting zu bringen, und so mehr Repräsentationskraft zu finden.

## In zwei Richtungen verlaufende RNNs
Das sind Modelle, die gerne im Natural Language Processing verwendet werden und mehr Performanz bieten.

In vielen Anwendungen, in denen die Reihenfolge von Daten von Bedeutung ist ex ante nicht klar, ob die Umkehr der Reihenfolge genauso gute Ergebnisse liefern kann. In Bereichen wie der Sprachverarbeitung funktioniert die Umkehrungen der Reihenfolge gut, denn die Bedeutung eines Wortes ist nicht nur abhängig von den vorangegangen Worten, sondern von der Position im gesamten Satz und somit auch von nachfolgenden Wörten. Aus diesem Grund kann das Umkehren der Sequenz sinnvoll sein und die sich daraus ergebenden Repräsentationen sollten genutzt werden.
Diese Überlegungen liegen den *bidirectional RNNs* zugrunde, welche sich ihre Inputs aus zwei Richtungen ansehen und beide daraus entstehenden Repäsentationen einbeziehen.

Technisch gesehen wird eine RNN-Schicht auf der chronologischen Reihenfolge trainiert und eine auf der antichronologischen Reihenfolge. 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, GRU, Embedding

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(Bidirectional(LSTM(32)))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1, activation = 'sigmoid'))

## Andere Wege zur Steigerung der Performanz
* Anpassen der Einheitenzahl in den recurrent Schichten
* Anpassen der Lernrate des `RMSprop`-Optimierers
* LSTM- statt GRU-Schichten
* größere Schichten von `Dense`-Schichten
* nicht vergessen, die besten Modelle auf dem Test-Set laufen zu lassen, da durch information leakage die Modelle auf dem Validierungssatz overfitten